In [84]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import seaborn as sns

# Set up
sub = 'SX'
save_fig = True

# Manage path
cur_dir = os.getcwd()
project_dir = os.path.dirname(os.path.dirname(cur_dir))
out_dir = os.path.join(cur_dir, 's1_plot_loc')
data_dir = os.path.join(project_dir, 'data', 'uniLoc')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [85]:
# Define a function to format the elements
def format_element(x):
    return float(f"{x[0][0]:.5f}")

In [86]:
# Clean
all_ses = ['A', 'V']

loc = []
estMu = []
sdMu = []
conf = []
err = []

# Define a function to format the elements
def format_element(x):
    return float(f"{x[0][0]:.5f}")

for s in all_ses:
    data = loadmat(os.path.join(data_dir, f'uniLoc_sub-{sub}_ses-{s}.mat'))  # Use loadmat to load .mat file
    ExpInfo = np.squeeze(data["ExpInfo"])
    sortedResp = np.squeeze(data['sortedResp'])

    nRep = int(ExpInfo['nRep'].squeeze())
    nLevel = int(ExpInfo['nLevel'].squeeze())

    # Localization data
    loc_rep = np.array(np.reshape(np.vectorize(format_element)(sortedResp['target_cm']), (nLevel, nRep)))
    loc.append(loc_rep[:, 0]) # Only take the first repetition because locations are the same for all
    est = np.array(np.reshape(np.vectorize(format_element)(sortedResp['response_cm']), (nLevel, nRep)))
    err.append(est - loc_rep)

    estMu.append(np.mean(est, axis=1))
    sdMu.append(np.std(est.astype(float), axis=1))  

    # Confidence data
    temp_conf = np.array(np.reshape(np.vectorize(format_element)(sortedResp['conf_radius_cm']), (nLevel, nRep)))
    conf.append(temp_conf)

loc = np.round(loc, 3)
estMu = np.round(estMu, 3)
sdMu = np.round(sdMu, 3)
conf = np.round(conf, 3)
err = np.round(err, 3)


## Line plot of localization responses

In [ ]:
# Plot
lw = 2
fontSZ = 15
titleSZ = 20
clt = np.array([[5, 113, 176], [202, 0, 32]]) / 255  # red and blue

plt.figure()
limax = 70

for i, s in enumerate(all_ses):
    plt.plot([-limax, limax], [-limax, limax], 'k--', linewidth=lw/2)
    e = plt.errorbar(loc[i], estMu[i], yerr=sdMu[i], color=clt[i], linewidth=lw, label=f'Session {s}')
    e[0].set_color(clt[i]) 

plt.xlim([-limax, limax])
plt.ylim([-limax, limax])
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('Target location (cm)', fontsize=fontSZ)
plt.ylabel('Estimated location (cm)', fontsize=fontSZ)

# Add legend
plt.legend()

# Remove the upper and right axis
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add the subject identifier at the right bottom corner
plt.text(0.95, 0.03, sub, horizontalalignment='right', verticalalignment='bottom', transform=ax.transAxes, fontsize=fontSZ)

if save_fig:
    fig_path = os.path.join(out_dir, f'{sub}_uni_loc.png')
    plt.savefig(fig_path, bbox_inches='tight')

plt.show()

## Violin plot of the confidence judgements

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Create a 1x2 subplot

for idx, s in enumerate(all_ses):
    session_index = all_ses.index(s)
    conf_s = conf[idx]

    sns.violinplot(data=[c.flatten() for c in conf_s], inner=None, linewidth=1, palette=[clt[idx]], ax=axes[idx])

    # Overlay raw data points
    for i, c in enumerate(conf_s):
        axes[idx].scatter([i] * len(c.flatten()), c.flatten(), color='black', s=10, alpha=0.6)


    # Use loc[0] for session 's' as x-axis labels
    axes[idx].set_xticks(range(len(loc[0])))
    axes[idx].set_ylim([0, 40])
    axes[idx].set_xlabel('Target location (cm)', fontsize=fontSZ)
    axes[idx].set_ylabel('Confidence Judgement (cm)', fontsize=fontSZ)
    axes[idx].spines['top'].set_visible(False)
    axes[idx].spines['right'].set_visible(False)
    axes[idx].set_title('Auditory', fontsize=titleSZ)
    axes[idx].set_title(f'Session {s}', fontsize=titleSZ)
    if idx == 0:
        axes[idx].text(0.1, 0.97, sub, horizontalalignment='right', verticalalignment='top', transform=axes[idx].transAxes, fontsize=fontSZ)


plt.tight_layout()
if save_fig:
    fig_path = os.path.join(out_dir, f'{sub}_uni_conf.png')
    plt.savefig(fig_path, bbox_inches='tight')

plt.show()

## Examine metacognitive sensitivity of unimodal localization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Create a 1x2 subplot

for idx, s in enumerate(all_ses):
    session_index = all_ses.index(s)
    err_s = np.abs(err[idx].flatten())
    conf_s = conf[idx].flatten()

    sns.scatterplot(x=err_s, y=conf_s, ax=axes[idx], color=clt[idx])
    sns.regplot(x=err_s, y=conf_s, ax=axes[idx], scatter=False, color=clt[idx])

    axes[idx].set_xlabel('Error (cm)', fontsize=fontSZ)
    axes[idx].set_ylabel('Confidence Judgement (cm)', fontsize=fontSZ)
    axes[idx].spines['top'].set_visible(False)
    axes[idx].spines['right'].set_visible(False)
    axes[idx].set_title(f'Session {s}', fontsize=titleSZ)
    if idx == 0:
        axes[idx].text(0.1, 0.97, sub, horizontalalignment='right', verticalalignment='top', transform=axes[idx].transAxes, fontsize=fontSZ)

plt.tight_layout()
if save_fig:
    fig_path = os.path.join(out_dir, f'{sub}_err_conf_corr.png')
    plt.savefig(fig_path, bbox_inches='tight')

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=False) 

# Define the color palette
palette = sns.color_palette("husl", 2)

for idx, s in enumerate(all_ses):
    session_index = all_ses.index(s)
    err_s = np.abs(err[idx].flatten())
    conf_s = conf[idx].flatten()

    # Determine the threshold for better and worse trials
    threshold = np.median(err_s)

    # Separate trials based on error size
    better_trials = conf_s[err_s <= threshold]
    worse_trials = conf_s[err_s > threshold]

    # Plot histogram with density curve for better trials
    sns.histplot(better_trials, kde=True, color=palette[0], ax=axes[idx], label='Better Trials', stat='density', bins=20, alpha=0.6, edgecolor=None)

    # Plot histogram with density curve for worse trials
    sns.histplot(worse_trials, kde=True, color=palette[1], ax=axes[idx], label='Worse Trials', stat='density', bins=20, alpha=0.6, edgecolor=None)

    axes[idx].set_xlabel('Confidence Judgement (cm)', fontsize=fontSZ)
    axes[idx].set_ylabel('Density', fontsize=fontSZ)
    axes[idx].spines['top'].set_visible(False)
    axes[idx].spines['right'].set_visible(False)
    axes[idx].set_title(f'Session {s}', fontsize=titleSZ)
    axes[idx].legend()

    # Set x-axis limits
    max_conf = np.max(conf_s)
    axes[idx].set_xlim(0, max_conf + 5)

    # Add participant ID to session A's panel
    if s == 'A':
        axes[idx].text(0.05, 0.95, sub, horizontalalignment='left', verticalalignment='top', transform=axes[idx].transAxes, fontsize=fontSZ)

plt.tight_layout()
if save_fig:
    fig_path = os.path.join(out_dir, f'{sub}_conf_hist.png')
    plt.savefig(fig_path, bbox_inches='tight')

plt.show()


In [ ]:
# Define the function to compute and plot ROC curve
def plot_roc_curve(better_trials, worse_trials, session_label):
    min_conf = min(min(better_trials), min(worse_trials))
    max_conf = max(max(better_trials), max(worse_trials))
    plt.gca().set_aspect('equal', adjustable='box')
    criteria = np.arange(min_conf, max_conf + 1, 1)

    hit_rates = []
    fa_rates = []

    for criterion in criteria:
        hits = np.sum(better_trials < criterion)
        false_alarms = np.sum(worse_trials < criterion)

        hit_rate = hits / len(better_trials)
        fa_rate = false_alarms / len(worse_trials)

        hit_rates.append(hit_rate)
        fa_rates.append(fa_rate)

    plt.plot(fa_rates, hit_rates, marker='o', label=f'Session {session_label}')
    plt.xlabel('False Alarm Rate')
    plt.ylabel('Hit Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)

# Plot ROC curves for both sessions
plt.figure(figsize=(10, 6))
for idx, s in enumerate(all_ses):
    session_index = all_ses.index(s)
    err_s = np.abs(err[idx].flatten())
    conf_s = conf[idx].flatten()

    # Determine the threshold for better and worse trials
    threshold = np.median(err_s)

    # Separate trials based on error size
    better_trials = conf_s[err_s <= threshold]
    worse_trials = conf_s[err_s > threshold]

    plot_roc_curve(better_trials, worse_trials, s)

# Add the subject identifier at the right bottom corner
plt.text(0.95, 0.03, sub, horizontalalignment='right', verticalalignment='bottom', transform=plt.gca().transAxes, fontsize=fontSZ)

if save_fig:
    fig_path = os.path.join(out_dir, f'{sub}_roc_curve.png')
    plt.savefig(fig_path, bbox_inches='tight')

plt.show()